<center><img src="./image/CLBLOGO.jpg" alt="创乐博" style="width: 300px;"/></center>

# 7.机器人视觉巡线

@－－－－湖南创乐博智能科技有限公司－－－－<br>
@  文件名：7.机器人视觉巡线.py <br>
@  版本：V2.0 <br>
@  author: zhulin<br>
@  说明：机器人视觉巡线
机器人基本运动！<br>

In [1]:
import numpy as np
import cv2
from LOBOROBOT import LOBOROBOT # 载入机器人库
import  RPi.GPIO as GPIO
import time 
import threading # 线程
import ctypes
import inspect

## 线程结束代码

In [2]:
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [3]:
# 按键及LED初始化
BtnPin  = 19
Gpin    = 5
Rpin    = 6

#初始化舵机
clbrobot = LOBOROBOT()  # 实例化机器人对象
clbrobot.t_stop(0)  # 停止

## 控制舵机的角度

In [4]:
# Configure min and max servo pulse lengths
servo_min = 150  # Min pulse length out of 4096
servo_max = 600  # Max pulse length out of 4096

# 频率设置为50hz，适用于舵机系统。
clbrobot.set_servo_angle(10,90)  # 底座舵机 90 
clbrobot.set_servo_angle(9,50)  # 顶部舵机 145
time.sleep(0.5)


In [5]:
# 按键控制函数
def keysacn():
    val = GPIO.input(BtnPin)
    while GPIO.input(BtnPin) == False:
        val = GPIO.input(BtnPin)
    while GPIO.input(BtnPin) == True:
        time.sleep(0.01)
        val = GPIO.input(BtnPin)
        if val == True:
            GPIO.output(Rpin,1)
            while GPIO.input(BtnPin) == False:
                GPIO.output(Rpin,0)
        else:
            GPIO.output(Rpin,0)

In [6]:
# 载入显示库
import ipywidgets.widgets as widgets
from IPython.display import display
image = widgets.Image(format='jpeg', width=160, height=120)
display(image)

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

Image(value=b'', format='jpeg', height='120', width='160')

In [7]:
cam = cv2.VideoCapture(0,cv2.CAP_V4L2)
#cam.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
#cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cam.set(3,160) 
cam.set(4,120) 
cam.isOpened()

True

In [8]:
GPIO.setwarnings(False) 
GPIO.setmode(GPIO.BCM)
GPIO.setup(Gpin, GPIO.OUT)     # 设置绿色Led引脚模式输出
GPIO.setup(Rpin, GPIO.OUT)     # 设置红色Led引脚模式输出
GPIO.setup(BtnPin, GPIO.IN, pull_up_down=GPIO.PUD_UP)    # 设置输入BtnPin模式，拉高至高电平(3.3V) 

In [9]:
def Video_display():
    # 键盘控制函数
    keysacn()
    while 1:
        # Capture the frames
        ret, frame = cam.read()
        frame=cv2.flip(frame, 1) 
        # Crop the image
        crop_img =frame[60:120, 0:160]
        # Convert to grayscale
        gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        # Gaussian blur
        blur = cv2.GaussianBlur(gray,(5,5),0)
        # Color thresholding
        ret,thresh1 = cv2.threshold(blur,60,255,cv2.THRESH_BINARY_INV)
        # Erode and dilate to remove accidental line detections
        mask = cv2.erode(thresh1, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)
        
       # Find the contours of the frame
        contours,hierarchy = cv2.findContours(mask.copy(), 1, cv2.CHAIN_APPROX_NONE)
        # Find the biggest contour (if detected)
        if len(contours) > 0:
            c = max(contours, key=cv2.contourArea)
            M = cv2.moments(c)
            
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            
            cv2.line(crop_img,(cx,0),(cx,720),(255,0,0),1)
            cv2.line(crop_img,(0,cy),(1280,cy),(255,0,0),1)
            
            cv2.drawContours(crop_img, contours, -1, (0,255,0), 1)
            
            if cx >= 120:
                clbrobot.turnRight(30,0)                 
                print("Turn Left!")
            if cx < 120 and cx > 50:
                clbrobot.t_up(50,0)
                print("On Track!")
            if cx <= 50:
                clbrobot.turnLeft(30,0)   
                print("Turn Right")
        else:
            print("I don't see the line")   
            
        image.value = bgr8_to_jpeg(crop_img)

## 开启线程

In [10]:
t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()

On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
Turn Right
Turn Right
Turn Right
Turn Right
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
Turn Right
Turn Right
Turn Right
Turn Right
Turn Right
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
Turn Right
Turn Right
Turn Right
Turn Right
Turn Right
Turn Right
Turn Right
Turn Right
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
On Track!
Turn Right
Turn Right
Turn Right
Turn Right
Turn Right
Turn Right
Turn Right
Turn R

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_2518/4216121750.py", line 7, in Video_display
NameError: name 'cv2' is not defined


## 关闭电机

In [12]:
clbrobot.t_stop(0)
GPIO.cleanup()            